# BLACKBOX
### Black Box es una funcionalidad fundamental utilizada para la búsqueda y descubrimiento de productos con potencial de rentabilidad y oportunidades dentro de Amazon. Su propósito principal es ayudar a los vendedores a filtrar grandes cantidades de datos para identificar nichos de mercado y productos que cumplen con criterios específicos.



### Importando librerías requeridas para el análisis

In [1]:
#%pip install IPython

import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots


### Archivo

### Leer el archivo BLACKBOX

In [2]:

blackbox = pd.read_csv('BLACKBOX.csv')

In [3]:
# Solo nos quedamos con las columnas importantes para el análisis

df = blackbox.drop(columns=['Image URL', 'BSR', 'UPC', 'GTIN', 'EAN', 'ISBN', 'Subcategory BSR',
                            'Price Trend (90 days) (%)', 'Parent Level Sales', 'Sales Trend (90 days) (%)',
                            'Seller Country/Region', 'Sales Year Over Year (%)', 'Length', 'Width', 'Height',
                            'Weight', 'Storage Fee (Jan - Sep)', 'Storage Fee (Oct - Dec)', 'Best Sales Period',
                            'Number of Active Sellers', 'Last Year Sales', 'Number of Images', 'Sales to Reviews'
                          ])

In [4]:
# Sustituit todos los '-' por cero
df = df.replace('-', 0)

# Convert 'ASIN Sales' and 'ASIN Revenue' to numeric, coercing errors
df['ASIN Sales'] = pd.to_numeric(df['ASIN Sales'], errors='coerce')
df['ASIN Revenue'] = pd.to_numeric(df['ASIN Revenue'], errors='coerce')

# Eliminamos todos los NaN sustituyéndolos por CERO

df['Reviews Rating'] = df['Reviews Rating'].fillna(0)
df['Reviews Rating'] = df['Reviews Rating'].astype(float)

df['Review Count'] = df['Review Count'].fillna(0)
df['Review Count'] = df['Review Count'].astype(int)

df['Variation Count'] = df['Variation Count'].fillna(0)
df['Variation Count'] = df['Variation Count'].astype(int)


### Aplicando los filtros:

In [5]:
altea = { "categories": ["Automotive", "Beauty & Personal Care", "Books", "CDs & Vinyl", "Cell Phones & Accessories",
                                "Clothing, Shoes & Jewelry", "Electronics", "Grocery & Gourmet Food", "Health & Household",
                                "Home & Kitchen", "Movies & TV", "Office Products", "Patio, Lawn & Garden", "Sports & Outdoors",
                                "Tools & Home Improvement", "Toys & Games"],
                 "reviewCountMax": 250,
                 "reviewRatingMax": 3.7,
                 "variationCountMax": 1,
                 "asinSalesMin": 100,
                 "asinRevenueMin": 4000,
                 "priceMin": 25,
                 "priceMax": 80,
                 "listingAgeMax": 18,
                 "sizeTier": ["Large bulky", "Small Standard-Size", "Large Standard-Size"],
                 "fulfillment": ["Amazon", "FBA", "FBM"] }

In [6]:
categories = df['Category'].isin(altea['categories'])
reviewCountMax = df['Review Count'] <= altea['reviewCountMax']
reviewRatingMax = df['Reviews Rating'] <= altea['reviewRatingMax']
variationCountMax = df['Variation Count'] <= altea['variationCountMax']
asinSalesMin = df['ASIN Sales'] >= altea['asinSalesMin']
asinRevenueMin = df['ASIN Revenue'] >= altea['asinRevenueMin']
priceMin = df['Price'] >= altea['priceMin']
priceMax = df['Price'] <= altea['priceMax']
listingAgeMax = df['Age (Month)'] <= altea['listingAgeMax']
sizeTier = df['Size Tier'].isin(altea['sizeTier'])
fulfillment = df['Fulfillment'].isin(altea['fulfillment'])

# Aplicando los filtros recomendados
df_filtrado = df[categories & reviewCountMax & reviewRatingMax & variationCountMax & asinSalesMin &
                 asinRevenueMin & priceMin & priceMax & listingAgeMax & sizeTier & fulfillment]

In [7]:
df_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 66 entries, 7 to 188
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   URL                   66 non-null     object 
 1   ASIN                  66 non-null     object 
 2   Title                 66 non-null     object 
 3   Brand                 59 non-null     object 
 4   Fulfillment           66 non-null     object 
 5   Category              66 non-null     object 
 6   Subcategory           43 non-null     object 
 7   Price                 66 non-null     float64
 8   ASIN Sales            66 non-null     int64  
 9   Parent Level Revenue  66 non-null     object 
 10  ASIN Revenue          66 non-null     float64
 11  Review Count          66 non-null     int64  
 12  Reviews Rating        66 non-null     float64
 13  Seller                66 non-null     object 
 14  Size Tier             66 non-null     object 
 15  Age (Month)           66 non-

### Análisis

In [8]:
# @title ASIN Revenue vs Review Count (ORIGINAL)

# Para mostrar la grafica de dispersión con los productos originales

fig = px.scatter(df, x='ASIN Revenue', y='Reviews Rating', size='Review Count', color='Fulfillment', hover_name='ASIN',
                 color_discrete_map={'Amazon': '#FFA15A', 'FBA': '#19D3F3', 'FBM':'#FF6692'},
                 category_orders={'Fulfillment': ['Amazon', 'FBA', 'FBM']},
                 custom_data=['URL'])

fig.update_layout(
    title='ASIN Revenue vs Review Count (ORIGINAL)',
    hovermode='closest',
    newshape=dict(line_color='black', line_width=2),
)

fig.show()

In [9]:
# @title ASIN Revenue vs Review Count (FILTRADA)

# Para mostrar la grafica de dispersión con los productos filtrados

fig = px.scatter(df_filtrado, x='ASIN Revenue', y='Reviews Rating', size='Review Count', color='Fulfillment', hover_name='URL',
                 color_discrete_map={'Amazon': '#FFA15A', 'FBA': '#19D3F3', 'FBM':'#FF6692'},
                 category_orders={'Fulfillment': ['Amazon', 'FBA', 'FBM']},
                 custom_data=['URL'])

fig.update_layout(
    title='ASIN Revenue vs Review Count (FILTRADA)',
    hovermode='closest',
    newshape=dict(line_color='black', line_width=2),
)

fig.show()